In [ ]:
# import libraries 
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import nltk
nltk.download('wordnet')
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

np.random.seed(400)

In [ ]:
def extracting_topics(data,x):
  '''
    Displays the required number of topics
    data: Pandas DataFrame or Series
    x: number of topics 
    '''
  tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
  dtm = tfidf.fit_transform(data)
  nmf_model = NMF(n_components=x,random_state=42)
  nmf_model.fit(dtm)

  # this view all the top 10 topics found.
  for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

In [ ]:
data2  = pd.read_csv("/content/drive/My Drive/Copy of data1.csv")
data2['content_message_local'] = data2['content_message_local'].apply(lambda x: str(x))
data2.content_message_local =data2.content_message_local.replace('nan',np.NAN)
data2.content_message_local = data2.content_message_local.astype(str)

In [ ]:
extracting_topics(data2.content_message_local,7)

THE TOP 15 WORDS FOR TOPIC #0
['hoehere', 'high', 'hemmung', 'hemmt', 'exposure', 'haq', 'guida', 'grenzen', 'greater', 'goals', 'gelenkschäden', 'für', 'funktionalitaet', 'gut', 'nan']


THE TOP 15 WORDS FOR TOPIC #1
['konsistent', 'hohe', 'bei', 'sicherheitsprofil', 'zeigt', 'vergleichbares', 'ein', 'vs', 'rinvoq', 'remissionsraten', 'adalimumab', 'hoehere', 'signifikant', 'erstmals', 'mtx']


THE TOP 15 WORDS FOR TOPIC #2
['als', 'für', 'patient', 'rinvoq', 'care', 'abbvie', 'psp', 'available', 'prescribed', 'patients', 'local', 'does', 'key', 'apply', 'message']


THE TOP 15 WORDS FOR TOPIC #3
['patientenpopulationen', 'hohe', 'allen', 'konsistent', 'bei', 'rinvoq', 'sicherheit', 'von', 'zulassungsstudien', 'wurde', 'umfangreichen', 'untersucht', 'die', 'grenzen', 'verschiebe']


THE TOP 15 WORDS FOR TOPIC #4
['dagewesene', 'di', 'rate', 'tassi', 'remissione', 'greater', 'significantly', 'rémission', 'et', 'vs', 'rates', 'ada', 'remission', 'placebo', 'mtx']


THE TOP 15 WORDS FOR 

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Import SentimentIntensityAnalyzer and create an sid object
from nltk.sentiment.vader import SentimentIntensityAnalyzer


def sentiment(string):

     '''Returns the sentiment of the data passed into it as (positive ,negative or neutral)
      string: pandas dataframe or series
    
    '''  
    sid = SentimentIntensityAnalyzer()
    # Obtain the sid scores 
    scores = sid.polarity_scores(string)
    if scores['compound'] == 0:
        return 'Neutral'
    elif scores['compound'] > 0:
        return 'Positive'
    else:
        return 'Negative'
    


In [ ]:
data2.apply(lambda x: sentiment(x.content_message_local), axis=1)

0         Neutral
1         Neutral
2         Neutral
3         Neutral
4         Neutral
           ...   
168528    Neutral
168529    Neutral
168530    Neutral
168531    Neutral
168532    Neutral
Length: 168533, dtype: object

In [ ]:
!pip install unidecode
import re
import string
import operator
import numpy as np
from unidecode import unidecode
from nltk import word_tokenize, sent_tokenize
from nltk import pos_tag_sents
from nltk.chunk.regexp import RegexpParser
from nltk.chunk import tree2conlltags
from nltk.corpus import stopwords
from itertools import chain, groupby
from operator import itemgetter
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
import nltk
nltk.download('averaged_perceptron_tagger')
punct_re = re.compile('[{}]'.format(re.escape(string.punctuation)))
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# phrase extraction

def generate_candidate(texts, method='word', remove_punctuation=False):
    """
    Generate word candidate from given string
    Parameters
    ----------
    texts: str, input text string
    method: str, method to extract candidate words, either 'word' or 'phrase'
    Returns
    -------
    candidates: list, list of candidate words
    """
    words_ = list()
    candidates = list()

    # tokenize texts to list of sentences of words
    sentences = sent_tokenize(texts)
    for sentence in sentences:
        if remove_punctuation:
            sentence = punct_re.sub(' ', sentence) # remove punctuation
        words = word_tokenize(sentence)
        words = list(map(lambda s: s.lower(), words))
        words_.append(words)
    tagged_words = pos_tag_sents(words_) # POS tagging

    if method == 'word':
        tags = set(['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'NNPS'])
        tagged_words = chain.from_iterable(tagged_words)
        for word, tag in tagged_words:
            if tag in tags and word.lower() not in stop_words:
                candidates.append(word)
    elif method == 'phrase':
        grammar = r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'
        chunker = RegexpParser(grammar)
        all_tag = chain.from_iterable([tree2conlltags(chunker.parse(tag)) for tag in tagged_words])
        for key, group in groupby(all_tag, lambda tag: tag[2] != 'O'):
            candidate = ' '.join([word for (word, pos, chunk) in group])
            if key is True and candidate not in stop_words:
                candidates.append(candidate)
    else:
        print("Use either 'word' or 'phrase' in method")
    return candidates

def keyphrase_extraction_tfidf(texts, method='phrase', min_df=5, max_df=0.8, num_key=5):
    """
    Use tf-idf weighting to score key phrases in list of given texts
    Parameters
    ----------
    texts: list, list of texts (remove None and empty string)
    Returns
    -------
    key_phrases: list, list of top key phrases that expain the article
    """
    print('generating vocabulary candidate...')
    vocabulary = [generate_candidate(unidecode(text), method=method) for text in texts]
    vocabulary = list(chain(*vocabulary))
    vocabulary = list(np.unique(vocabulary)) # unique vocab
    print('done!')
    # print(vocabulary)

    max_vocab_len = max(map(lambda s: len(s.split(' ')), vocabulary))
    tfidf_model = TfidfVectorizer(vocabulary=vocabulary, lowercase=True,
                                  ngram_range=(1,max_vocab_len), stop_words=None,
                                  min_df=min_df, max_df=max_df)
    X = tfidf_model.fit_transform(texts)
    print(tfidf_model)
    vocabulary_sort = [v[0] for v in sorted(tfidf_model.vocabulary_.items(),
                                            key=operator.itemgetter(1))]
    sorted_array = np.fliplr(np.argsort(X.toarray()))
    print(sorted_array)
    # return list of top candidate phrase
    key_phrases = list()
    for sorted_array_doc in sorted_array:
        key_phrase = [vocabulary_sort[e] for e in sorted_array_doc[0:num_key]]
        key_phrases.append(key_phrase)

    return key_phrases

In [ ]:
keyphrase_extraction_tfidf(data2.content_message_local)

generating vocabulary candidate...
done!
TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=5, ngram_range=(1, 16), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, us...
                            'adalimumab+mtx', 'adherence',
                            'ansprechen durch abbviecare', 'auf vielen ebenen',
                            'badan klinicznych',
                            'bei allen patientenpopulationen', 'besseres',
                            'bezpecnostni profil', 'bol', 'brand summary',
                            'clinical program', 'coinvolti',
                            'complete skin clearance', 'consistency',
                   

[['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['z i',
  'inhibice rtg progrese s/bez mtx',
  'il23 with durability',
  'il-23/p19',
  'humira + mtx'],
 ['nan', 'z i', 'haq-di vs ada + mtx', 'il23 with durability', 'il-23/p19'],
 ['z i',
  'inhibice rtg progrese s/bez mtx',
 